In [2]:
pip install ipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher
from ipynb.fs.full.helper import get_text_from_csv,read_fb,clean_text,check_null,find_words,find_topic,get_word_lable,count_topic,count_word

ModuleNotFoundError: No module named 'ipynb'

# GOOGLE + FACEBOOK

In [2]:
google_df = get_text_from_csv('GoogleAds/texas.csv')
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             148 non-null    object
 1   advertiser_name  148 non-null    object
 2   platform         148 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [3]:
google_df.head()

,text,advertiser_name,platform
0,"""Election Day: MARCH 1"",""Vote for a Brighter F...",JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google
1,"""Stand with Captain Sam Brown. Let's restore a...",SAM BROWN FOR NEVADA,google
2,"""Let's make voting simpler, more convenient, a...",Kurt Lockhart Campaign,google
3,"""Texas's 29th Congressional District",JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google
4,"""Let's make voting simpler, more convenient, a...",Kurt Lockhart Campaign,google


In [4]:
facebook_df = read_fb('FacebookAds/Texas.csv')
facebook_df

,text,advertiser_name,platform
0,"Elon Musk's 44 billion dollar cash offer for Twitter is accepted, so a new day dawns for communication in the public square ""for the future of humanity."" Now as one Republican officeholder that really started on Twitter after the former CEO Dorsey cut off ads for political accounts, I stayed on tweeting conservative public policy solutions to Texas' challenges! I wasn't going to give up on the public square discussion and slowly doubled my audience with good content @teambettencourt. So, I wish Elon Musk well in his already announced reforms. This Reuters article covers that and more, and It's well worth the read at:\nhttps://www.reuters.com/technology/exclusive-twitter-set-accept-musks-best-final-offer-sources-2022-04-25/\n\nThe best public policy stands out in any public debate, and Musk is smart enough to realize that standing up for free speech pays tremendous dividends to all those that participate. That's why he's not worried about the economics of his purchase publicly because if he does his job, he will exceed expectations again like he did at PayPal, Tesla, and SpaceX!!",Friends of Paul Bettencourt,facebook
1,"🚩 WTF: First it was Texas. And at least a DOZEN other states are at risk of passing a bill similar to Texas's extreme six-week abortion ban.\n\nThese cruel bans disproportionately harm Black & Latinx people, people with low incomes, and people in rural areas, who already face immense barriers to health care access. It's up to us to stop it >>",Planned Parenthood Action Fund,facebook
2,It is essential that Dems nominate our STRONGEST advocate to win in November. Mihaela Plesa is the only candidate in this race that has the experience to deliver results on day one!,Mihaela Plesa for Texas House,facebook
3,NaN,NaN,facebook
4,"Hey Bob Moore! What say you debate me?\n\nI challenge you to a debate on Facebook Live! this coming Friday, April 29th at 7pm.\n\n(Folks, if you’d like to see this debate, please share this post everywhere! 😊)",Shannon Thomason Campaign,facebook
...,...,...,...
28536,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28537,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28538,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28539,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook


In [5]:
# concat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text

,text,advertiser_name,platform
0,"""Election Day: MARCH 1"",""Vote for a Brighter Future !",JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google
1,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada.",SAM BROWN FOR NEVADA,google
2,"""Let's make voting simpler, more convenient, and more accessible"",""Vote Kurt for Clerk in the March 1 Democratic Primary!",Kurt Lockhart Campaign,google
3,"""Texas's 29th Congressional District",JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google
4,"""Let's make voting simpler, more convenient, and more accessible",Kurt Lockhart Campaign,google
...,...,...,...
28536,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28537,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28538,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook
28539,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93.",Laura Hill for Texas,facebook


In [6]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

16


In [7]:
df_text = clean_text(df_text)
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             387 non-null    object
 1   advertiser_name  371 non-null    object
 2   platform         387 non-null    object
dtypes: object(3)
memory usage: 9.2+ KB


/Users/yzhao/git/ml-political-ads/helper.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "id": "5b928802",
/Users/yzhao/git/ml-political-ads/helper.ipynb:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "metadata": {},
/Users/yzhao/git/ml-political-ads/helper.ipynb:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [8]:
df_text = get_word_lable(df_text,'lexicon.json')
df_text

/Users/yzhao/git/ml-political-ads/helper.ipynb:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    for t in topics:\n",
/Users/yzhao/git/ml-political-ads/helper.ipynb:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        matcher = PhraseMatcher(nlp.vocab)\n",
/Users/yzhao/git/ml-political-ads/helper.ipynb:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,text,advertiser_name,platform,words,m_label
0,election day march 1 vote for a brighter future,JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google,no words,no topic
1,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,SAM BROWN FOR NEVADA,google,no words,no topic
2,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary,Kurt Lockhart Campaign,google,no words,no topic
3,texas s 29th congressional district,JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,google,no words,no topic
4,let s make voting simpler more convenient and more accessible,Kurt Lockhart Campaign,google,no words,no topic
...,...,...,...,...,...
382,transportation roads and traffic are the top issues for the folks in hd93,Laura Hill for Texas,facebook,transportation,economic
383,i championed school safety and put a school resource officer in every carroll isd school to keep students safe defund the police radicals tried to remove those officers but i fought back and won i refuse to back down when the safety of our families and students is on the line i m a proven conservative fighter and i ll get the job done for us here in hd93,Laura Hill for Texas,facebook,"school,police,radicals,officers","education,safety,social and cultural"
384,as the former southlake mayor i have firsthand experience fighting back against the critical race theory agenda that pits neighbor against neighbor the american dream is open and accessible to all who are willing to work hard that starts with investing in a great education for our students keeping politicized curriculums out of the classroom and ensuring our schools stay open for in person learning click and stand with me for a better education system today,Laura Hill for Texas,facebook,education,education
385,laura hill has fought back against critical race theory from day 1 and she has the battle scars to prove it parents should have the final say in their children s education and we have spoken no crt and no liberal political agendas in our schools,Laura Hill for Texas,facebook,education,education


In [9]:
 df_text.to_csv('results/ad_tx.csv')

In [10]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,178
1,economic,63
2,safety,41
3,immigration,38
4,social and cultural,38
5,education,36
6,donation,26
7,governance,24
8,health,24
9,environment,20


In [11]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,178
1,border,28
2,taxes,22
3,school,16
4,tax,16
...,...,...
199,graduates,1
200,gun,1
201,illegal immigration,1
202,immigration law,1


# top 5 advertiser

In [16]:
df_text['advertiser_name'].value_counts().head(5)

Beto for Texas                 14
Laura Hill for Texas           13
COLLIER COMMITTEE              11
Democracy Initiative            7
BETH VAN DUYNE FOR CONGRESS     7
Name: advertiser_name, dtype: int64

In [18]:
beto = df_text[df_text['advertiser_name']== "Beto for Texas"]
beto

,text,advertiser_name,words,m_label
78,donate 15 or more and we will send you a union made sign,Beto for Texas,donate,donation
148,earlier this month greg abbott traded 4 2 billion in economic devastation for a 3 minute and 20 second interview on fox news his political stunt cost texans their jobs and worsened inflation across this state and the country as he jacked up prices at the grocery store he cannot be allowed to serve another term i promise that as governor i will put texas first every single time while creating jobs instead of killing them we are building the largest grassroots movement this state has ever seen can you chip in 10 right now to help us reach our goal and fire greg abbott,Beto for Texas,"inflation,jobs,economic,creating jobs,chip in","donation,economic"
149,i m running for governor to represent all the people of texas which means i don t accept a cent of corporate pac money with hundreds of thousands of donations to this campaign so far and an average contribution of 39 71 this campaign proves just how powerful grassroots fundraising is we set a goal of raising 1 000 000 before the end of the month and it is critical that we do not fall a dollar behind can you chip in 10 or 20 today to join our grassroots movement,Beto for Texas,"chip in,contribution",donation
151,we have set an ambitious goal to raise 1 000 000 by the end of this month we re going to have to hustle to hit this goal and secure the resources we need to compete with greg abbott s more than 50 million in the bank with the future of texas on the line we can t risk falling short 𝐖𝐡𝐢𝐥𝐞 𝐀𝐛𝐛𝐨𝐭𝐭 𝐟𝐮𝐧𝐝𝐬 𝐡𝐢𝐬 𝐜𝐚𝐦𝐩𝐚𝐢𝐠𝐧 𝐰𝐢𝐭𝐡 𝐜𝐨𝐫𝐩𝐨𝐫𝐚𝐭𝐞 𝐜𝐡𝐞𝐜𝐤 𝐚𝐟𝐭𝐞𝐫 𝐜𝐨𝐫𝐩𝐨𝐫𝐚𝐭𝐞 𝐜𝐡𝐞𝐜𝐤 𝐰𝐞 𝐫𝐞 𝐩𝐫𝐨𝐮𝐝 𝐭𝐨 𝐬𝐚𝐲 𝐭𝐡𝐚𝐭 𝐰𝐞 𝐯𝐞 𝐫𝐞𝐜𝐞𝐢𝐯𝐞𝐝 𝐡𝐮𝐧𝐝𝐫𝐞𝐝𝐬 𝐨𝐟 𝐭𝐡𝐨𝐮𝐬𝐚𝐧𝐝𝐬 𝐨𝐟 𝐝𝐨𝐧𝐚𝐭𝐢𝐨𝐧𝐬 𝐭𝐨 𝐭𝐡𝐢𝐬 𝐜𝐚𝐦𝐩𝐚𝐢𝐠𝐧 we don t accept a cent from corporate pacs so it s donations of 5 10 and 15 that power this campaign we re counting on this movement to help us prepare for every single attack that will come our way and to spread our positive unifying vision for texas across the state can you chip in today to help us reach our fundraising goal and elect beto as the next governor of texas,Beto for Texas,chip in,donation
152,the stakes could not be higher in this election under greg abbott s failed leadership texas has adopted some of the most radical harmful and cruel policies in the entire country and our families are paying the price as governor beto will get to work on the issues that actually matter to texans like reducing health care costs investing in our public schools and bringing good jobs to this state we set an ambitious goal to raise 1 000 000 by the end of the month can you chip in 15 or 20 to ensure that we have the resources we need to go the distance and win this election,Beto for Texas,"jobs,health,care costs,public schools,chip in,health care","donation,economic,education,health"
194,what if instead of having a governor who s focused on stunts that jack up inflation we had a governor who s focused on solutions that reduce your property taxes,Beto for Texas,"taxes,inflation",economic
229,i m going to ask you to donate to our grassroots campaign but first let me tell you why it is so important to contribute at this exact moment first we officially won our primary campaign on march 1st with over 90 of the vote then we immediately began a general election fight with greg abbott who has over 50 million in the bank and billionaire donors who can cut him massive checks whenever he calls now we re building the biggest volunteer organizing team in the country and these volunteers are already at the doors of texas voters building relationships and connecting with those who will decide the outcome of this election chipping in now ensures that we will have the resources to hire more organizers run ads and reach every single eligible voter in texas with our unifying message can you donate today to help us and defeat abbott,Beto for Texas,"chipping in,donate,contribute",donation
230,43 07 that represents the historic p

In [19]:
count_topic(beto)

,single_topic,counts
0,donation,13
1,economic,6
2,health,5
3,social and cultural,4
4,safety,3
5,education,2
6,social programs,1


In [22]:
count_word(beto)

,single_word,counts
0,chip in,6
1,donate,5
2,health,5
3,health care,4
4,economic,3
5,marijuana,3
6,jobs,3
7,reproductive rights,2
8,racial,2
9,inflation,2


In [23]:
laura_hill = df_text[df_text['advertiser_name']== "Laura Hill for Texas"]
laura_hill

,text,advertiser_name,words,m_label
156,vote laura hill for house district 93,Laura Hill for Texas,no words,no topic
157,laura hill will protect us from liberals who want to force small businesses to comply with their virtue signaling covid mandates,Laura Hill for Texas,"covid,small businesses","covid,economic"
158,when covid closed the door to the classroom the door opened for parents to see what was going on in our schools vote laura hill for hd 93 laura will make sure parents have the final say in their children s education and she will keep liberal agendas away from our kids education,Laura Hill for Texas,"education,covid","covid,education"
159,laura hill proven track record of supporting the police and prioritizing public safety we need laura hill,Laura Hill for Texas,"police,public safety",safety
378,proud to be endorsed by republican pat hardy she fights every day for our conservative values,Laura Hill for Texas,no words,no topic
379,big announcement on occasion there are moments in life that feel like god is saying yes you are on the right path today when i received word this endorsement was coming later today i looked over at joe and had that moment this endorsement from the fort worth police officers association means everything to me my record is clear i have spent over 20 years in public service and have always fought for our police fire and emergency response teams once in austin i will show the same commitment and will fight anyone who dares approach me with legislation that even sniffs of defunding the police reducing their staff or taking away their resources every day these brave men and women get up and put on their badge and in doing so they put their life on the line for each of us hd 93 will always backtheblue txlege hd93 experiencematters fort worth police department fort worth republican women cowtown republican women metroplex republican women tarrant county republican party life pac texas federation of republican women,Laura Hill for Texas,"police officers,police department,police,officers",safety
380,honored to be endorsed by mona bailey mona has set the standard for hard working republican women in tarrant county she has been a mentor to so many and a fighter for our conservative values a woman i greatly admire www laurahillfortexas com join us earnit workinghardeveryday hd93 republicanrunoff txlege campaign texashousedistrict93,Laura Hill for Texas,no words,no topic
381,honored to have the endorsement of sharen wilson the first woman to serve as criminal district attorney in tarrant county a friend and mentor,Laura Hill for Texas,criminal,safety
382,transportation roads and traffic are the top issues for the folks in hd93,Laura Hill for Texas,transportation,economic
383,i championed school safety and put a school resource officer in every carroll isd school to keep students safe defund the police radicals tried to remove those officers but i fought back and won i refuse to back down when the safety of our families and students is on the line i m a proven conservative fighter and i ll get the job done for us here in hd93,Laura Hill for Texas,"radicals,school,police,officers","education,safety,social and cultural"


In [24]:
count_topic(laura_hill)

,single_topic,counts
0,education,4
1,safety,4
2,economic,3
3,no topic,3
4,covid,2
5,social and cultural,1


In [25]:
count_word(laura_hill)

,single_word,counts
0,education,3
1,no words,3
2,police,3
3,covid,2
4,officers,2
5,criminal,1
6,police department,1
7,police officers,1
8,public safety,1
9,radicals,1


In [26]:
collier = df_text[df_text['advertiser_name']== "COLLIER COMMITTEE"]
collier

,text,advertiser_name,words,m_label
40,vote for mike collier in the democratic primary for texas lieutenant governor mike will defend voting rights protect the right to choose and fix the damn grid,COLLIER COMMITTEE,"right to choose,voting rights","governance,social and cultural"
49,mike collier is running for lieutenant governor to defeat dan patrick and fight for texas,COLLIER COMMITTEE,no words,no topic
68,mike will defend voting rights protect the right to choose and fix the damn grid vote for mike collier in the democratic primary for texas lieutenant governor,COLLIER COMMITTEE,"right to choose,voting rights","governance,social and cultural"
71,voting in the democratic primary is happening now cast your vote for mike today,COLLIER COMMITTEE,no words,no topic
86,mike will defend voting rights protect the right to choose and fix the damn grid,COLLIER COMMITTEE,"right to choose,voting rights","governance,social and cultural"
121,the only thing standing between me and dan patrick is the money i need to get our message out as democrats will you chip in today and help us reach our end of month fundraising goal,COLLIER COMMITTEE,chip in,donation
123,the only thing standing between me and dan patrick is the money i need to get our message out as democrats our state is not a prop in greg abbott and dan patrick s political theater yet abbott and patrick have spent years trying to swindle us by using fear of our neighbors and our government to divert our attention from their inability to deliver for the people of texas 100 million per day that is how much greg abbott s latest border stunt is costing our economy governance by stunts all to distract from our rickety power grid our skyrocketing property taxes and our fundamental rights hanging in the balance well folks i ve had enough and i think the majority of texans have had enough we deserve better this november we can hold them accountable and beat them but we ll need money and your support to get the job done we re coming up on our last end of month fundraising deadline before the democratic primary so anything you can give right now will help us lay the groundwork for a tough fight and allow us to win i mean that will you chip in today and help us reach our end of month fundraising goal,COLLIER COMMITTEE,"chip in,economy,taxes,border","donation,economic,immigration"
124,let s defeat the worst lt governor in the nation will you donate before our end of month deadline to make sure we can beat dan patrick this november,COLLIER COMMITTEE,donate,donation
125,100 million per day that is how much greg abbott s latest border stunt is costing our economy our state is not a prop in greg abbott and dan patrick s political theater yet abbott and patrick have spent years trying to swindle us by using fear of our neighbors and our government to divert our attention from their inability to deliver for the people of texas governance by stunts all to distract from our rickety power grid our skyrocketing property taxes and our fundamental rights hanging in the balance well folks i ve had enough we deserve better this november we can hold them accountable and beat them and i am pumped to do it but we cannot do it without your all important support we re coming up on our last end of month fundraising deadline before the democratic primary so anything you can give right now will help us lay the groundwork for a tough fight and allow us to win i mean that,COLLIER COMMITTEE,"economy,taxes,border","economic,immigration"
126,the only thing standing between me and the worst lt governor in the nation is the money i need to get our message out greg abbott and dan patrick have spent years trying to swindle us by using fear of our neighbors and our government to divert our attention from their inability to deliver for the people of texas and their latest border stunt is costing our economy 100 million per day i ve had enough and i think the majority of texans have had enough we deserve better this november w

In [27]:
count_topic(collier)

,single_topic,counts
0,donation,5
1,economic,3
2,governance,3
3,immigration,3
4,social and cultural,3
5,no topic,2


In [28]:
count_word(collier)

,single_word,counts
0,border,3
1,chip in,3
2,economy,3
3,right to choose,3
4,voting rights,3
5,donate,2
6,no words,2
7,taxes,2


In [29]:
demo = df_text[df_text['advertiser_name']== "Democracy Initiative"]
demo

,text,advertiser_name,words,m_label
29,if we want action on the problems facing america we must fix or nix the filibuster mcconnell enriches his wealthy patrons and uses the filibuster to block everything else,Democracy Initiative,filibuster,governance
33,the filibuster stands in the way we need to fix or nix the filibuster you can help,Democracy Initiative,filibuster,governance
38,states are passing laws to silence voters federal standards would put a stop to it,Democracy Initiative,no words,no topic
60,the filibuster is used to block protections for our democracy it s time to fix or nix it it s been a year since january 6 but the senate hasn t acted to protect our elections,Democracy Initiative,filibuster,governance
61,it s been a year since january 6 but the senate hasn t acted to protect our elections the filibuster is used to block protections for our democracy it s time to fix or nix it,Democracy Initiative,filibuster,governance
80,there s no greater priority than defending democracy you can help,Democracy Initiative,no words,no topic
81,the january 6 insurrection kicked off a concerted wave of anti voting laws nationwide there s no greater priority than defending democracy you can help,Democracy Initiative,no words,no topic


In [31]:
beth_van = df_text[df_text['advertiser_name']=='BETH VAN DUYNE FOR CONGRESS']
beth_van

,text,advertiser_name,words,m_label
13,we can put an end to socialism and tyranny with your help get to know beth and her fight to secure our borders u0026 put americans first,BETH VAN DUYNE FOR CONGRESS,borders,immigration
35,beth van duyne is running for re election in congress tx 24 and needs your support donate to trump endorsed republican beth van duyne for congress,BETH VAN DUYNE FOR CONGRESS,donate,donation
42,texas needs to keep a conservative fighter in the house who puts america first,BETH VAN DUYNE FOR CONGRESS,no words,no topic
55,beth van duyne is running for re election in congress tx 24 and is trump endorsed republican beth van duyne is running for tx 24 and is endorsed by president donald trump,BETH VAN DUYNE FOR CONGRESS,no words,no topic
69,contribute to beth van duyne u0026 help her fight to secure our borders u0026 put americans first we can put an end to socialism and tyranny with your support,BETH VAN DUYNE FOR CONGRESS,"borders,contribute","donation,immigration"
72,help texas keep a conservative fighter in the house who puts america first support a fighter in congress who is up against nancy pelosi and the radical left,BETH VAN DUYNE FOR CONGRESS,no words,no topic
355,just a quick note of thanks to the heroic men and women of our border patrol for their continued sacrifices and determination to protect our country from evil drug cartels and human traffickers who are driving poison into our communities and waves of mass illegal immigration across our borders the courage and resilience of our border patrol is beyond commendable especially as they wage a daily struggle to protect america despite how they are attacked lied about and handcuffed by the biden administration and the radicals who control congress to the men and women of cbp ice and dps we thank you lift you up in our prayers and solemnly pledge help is coming this november hold the line as best you can we are with you thank you national border patrol council and texas department of public safety,BETH VAN DUYNE FOR CONGRESS,"borders,radicals,border,immigration,public safety,illegal immigration","immigration,safety,social and cultural"


# GOOGLE

In [13]:
if google_df['text'].isnull().sum():
    print (google_df['text'].isnull().sum())
    google_df = google_df[google_df['text'].notnull()]
    google_df.reset_index(drop=True,inplace=True)

In [14]:
google_df = clean_text(google_df)
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             90 non-null     object
 1   advertiser_name  90 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [15]:
google_df = get_word_lable(google_df,'lexicon.json')
google_df.head()

,text,advertiser_name,words,m_label
0,election day march 1 vote for a brighter future,JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,no words,no topic
1,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,SAM BROWN FOR NEVADA,no words,no topic
2,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary,Kurt Lockhart Campaign,no words,no topic
3,texas s 29th congressional district,JAIMY Z. BLANCO FOR CONGRESS CAMPAIGN,no words,no topic
4,let s make voting simpler more convenient and more accessible,Kurt Lockhart Campaign,no words,no topic


In [10]:
google_topic = count_topic(google_df)
google_topic

KeyError: 'm_label'

In [ ]:
google_word = count_word(google_df)
google_word

# FACEBOOK 

In [ ]:
if facebook_df['text'].isnull().sum():
    print (facebook_df['text'].isnull().sum())
    facebook_df = facebook_df[facebook_df['text'].notnull()]
    facebook_df.reset_index(drop=True,inplace=True)
facebook_df = clean_text(facebook_df)
facebook_df = get_word_lable(facebook_df,'lexicon.json')

In [ ]:
facebook_topic = count_topic(facebook_df)
facebook_topic

In [ ]:
facebook_word = count_word(facebook_df)
facebook_word